In [1]:
%matplotlib inline


SortingExtractor objects
========================

The :code:`SortingExtractor` is the basic class for handling spike sorted data. Here is how it works.



In [2]:
import numpy as np
import spikeinterface.extractors as se

We will create a :code:`SortingExtractor` object from scratch using :code:`numpy` and the
:code:`NumpySortingExtractor`

Let's define the properties of the dataset



In [3]:
sampling_frequency = 30000
duration = 20
num_timepoints = int(sampling_frequency * duration)
num_units = 4
num_events = 1000

We generate some random events.



In [4]:
times = np.int_(np.sort(np.random.uniform(0, num_timepoints, num_events)))
labels = np.random.randint(1, num_units + 1, size=num_events)

And instantiate a :code:`NumpyRecordingExtractor`:



In [5]:
sorting = se.NumpySortingExtractor()
sorting.set_times_labels(times=times, labels=labels)
sorting.set_sampling_frequency(sampling_frequency=sampling_frequency)

We can now print properties that the :code:`SortingExtractor` retrieves from the underlying sorted dataset.



In [6]:
print('Unit ids = {}'.format(sorting.get_unit_ids()))
st = sorting.get_unit_spike_train(unit_id=1)
print('Num. events for unit 1 = {}'.format(len(st)))
st1 = sorting.get_unit_spike_train(unit_id=1, start_frame=0, end_frame=30000)
print('Num. events for first second of unit 1 = {}'.format(len(st1)))

Unit ids = [1, 2, 3, 4]
Num. events for unit 1 = 249
Num. events for first second of unit 1 = 15


Some extractors also implement a :code:`write` function. We can for example save our newly created sorting into
MDA format (Mountainsort4 format):



In [7]:
se.MdaSortingExtractor.write_sorting(sorting=sorting, save_path='firings_true.mda')

and read it back with the proper extractor:



In [8]:
sorting2 = se.MdaSortingExtractor(file_path='firings_true.mda',
                                  sampling_frequency=sampling_frequency)
print('Unit ids = {}'.format(sorting2.get_unit_ids()))
st = sorting2.get_unit_spike_train(unit_id=1)
print('Num. events for unit 1 = {}'.format(len(st)))
st1 = sorting2.get_unit_spike_train(unit_id=1, start_frame=0, end_frame=30000)
print('Num. events for first second of unit 1 = {}'.format(len(st1)))

Unit ids = [1, 2, 3, 4]
Num. events for unit 1 = 249
Num. events for first second of unit 1 = 15


Unit properties are name value pairs that we can store for any unit. We will now calculate a unit
property and store it in the :code:`SortingExtractor`



In [9]:
full_spike_train = sorting2.get_unit_spike_train(unit_id=1)
firing_rate = float(len(full_spike_train)) / sorting2.get_sampling_frequency()
sorting2.set_unit_property(unit_id=1, property_name='firing_rate', value=firing_rate)
print('Average firing rate during the recording of unit 1 = {}'.format(sorting2.get_unit_property(unit_id=1,
                                                                                                  property_name='firing_rate')))
print("Spike property names: " + str(sorting2.get_shared_unit_property_names()))

Average firing rate during the recording of unit 1 = 0.0083
Spike property names: ['max_channel']


:code:`SubSortingExtractor` objects can be used to extract arbitrary subsets of your units/spike trains manually



In [10]:
sorting3 = se.SubSortingExtractor(parent_sorting=sorting2, unit_ids=[1, 2],
                                  start_frame=10000, end_frame=20000)
print('Num. units = {}'.format(len(sorting3.get_unit_ids())))
print('Average firing rate of units1 during frames 10000-20000 = {}'.format(
    float(len(sorting3.get_unit_spike_train(unit_id=1))) / 6000))

Num. units = 2
Average firing rate of units1 during frames 10000-20000 = 0.0008333333333333334


Unit features are name value pairs that we can store for each spike. Let's load a randomly generated 'random_value'
features. Features are used, for example, to store waveforms, amplitude, and PCA scores



In [11]:
random_values = np.random.randn(len(sorting3.get_unit_spike_train(unit_id=1)))
sorting3.set_unit_spike_features(unit_id=1, feature_name='random_value',
                                 value=random_values)
print("Spike feature names: " + str(sorting3.get_shared_unit_spike_feature_names()))

Spike feature names: []
